**BBO (Best Bid and Offer)** — это данные, которые отображают лучшую цену покупки (bid) и лучшую цену продажи (ask) на рынке в конкретный момент времени.

bbo_1000pepeusdt.csv и bbo_dogeusdt.csv
 
1. local_timestamp: Время, когда было зафиксировано состояние рынка, выраженное в локальном времени. 
2. ask_amount: Количество актива, которое доступно на продажу по лучшей (минимальной) цене продажи (ask_price).
3. ask_price: Лучшая цена продажи (цена, по которой кто-то готов продать актив).
4. bid_price: Лучшая цена покупки (цена, по которой кто-то готов купить актив).
5. bid_amount: Количество актива, которое доступно на покупку по лучшей (максимальной) цене покупки (bid_price).


**Trades (сделки)** — это данные о совершенных сделках на рынке.

trades_1000pepeusdt.csv и trades_dogeusdt.csv

1. local_timestamp: Время, когда была совершена сделка.
2. side: Направление сделки, указывающее, была ли сделка на покупку (buy) или продажу (sell).
3. price: Цена, по которой была совершена сделка.
4. amount: Количество актива, которое было куплено или продано в сделке.

По этим таблицам формируем свечи OLHC

**Свечи (OLHC candles)**

1. *Open*: цена первой сделки в окне
2. *Close*: цена последней сделки в окне
3. *Low*: минимальная цена сделки в окне
4. *High*: максимальная цена сделки в окне
5. *Средняя цена покупок*: среднее арифметическое всех цен сделок на покупку в окне
6. *Средняя цена продаж*: среднее арифметическое всех цен сделок на продажу в окне
7. *Объем покупок и продаж*: сумма объемов всех сделок на покупку и продажу в окне

In [31]:
import pandas as pd

from datetime import datetime


#загрузка ордербука

bbo_1000pepeusdt=pd.read_csv('/Users/anastasiabondarenko/Downloads/md_sim-2/bbo_1000pepeusdt.csv')

bbo_dogeusdt=pd.read_csv('/Users/anastasiabondarenko/Downloads/md_sim-2/bbo_dogeusdt.csv')




#загрузка трейдов

trades_1000pepeusdt=pd.read_csv('/Users/anastasiabondarenko/Downloads/md_sim-2/trades_dogeusdt.csv')

trades_dogeusdt=pd.read_csv('/Users/anastasiabondarenko/Downloads/md_sim-2/trades_dogeusdt.csv')

 

#корректировка времени

bbo_1000pepeusdt['timestamp']=pd.to_datetime(bbo_1000pepeusdt['local_timestamp'], unit='us')

bbo_dogeusdt['timestamp']=pd.to_datetime(bbo_dogeusdt['local_timestamp'], unit='us')

trades_1000pepeusdt['timestamp']=pd.to_datetime(trades_1000pepeusdt['local_timestamp'], unit='us')

trades_dogeusdt['timestamp']=pd.to_datetime(trades_dogeusdt['local_timestamp'], unit='us')



#индексы -> время

bbo_1000pepeusdt.set_index('timestamp',inplace=True)

bbo_dogeusdt.set_index('timestamp',inplace=True)

trades_1000pepeusdt.set_index('timestamp',inplace=True)

trades_dogeusdt.set_index('timestamp',inplace=True)
 

T='30Min'


#создание OHLC свечей для pepeusdt


pepeusdt_ohlc = trades_1000pepeusdt['price'].resample(T).ohlc()

#объем покупок и продаж 

trades_1000pepeusdt['product']=trades_1000pepeusdt['price']*trades_1000pepeusdt['amount']

#cредняя цена покупок

agg_trades_1000pepeusdt_buy=trades_1000pepeusdt[trades_1000pepeusdt['side']=='buy'].groupby(pd.Grouper(freq=T))[['product','amount']].sum()

agg_trades_1000pepeusdt_buy['avg_price']=agg_trades_1000pepeusdt_buy['product']/agg_trades_1000pepeusdt_buy['amount']

agg_trades_1000pepeusdt_buy=agg_trades_1000pepeusdt_buy[['amount','avg_price']].rename(columns={'amount':'traded'}).add_suffix('_buy')

#средняя цена продаж

agg_trades_1000pepeusdt_sell=trades_1000pepeusdt[trades_1000pepeusdt['side']=='sell'].groupby(pd.Grouper(freq=T))[['product','amount']].sum()

agg_trades_1000pepeusdt_sell['avg_price']=agg_trades_1000pepeusdt_sell['product']/agg_trades_1000pepeusdt_sell['amount']

agg_trades_1000pepeusdt_sell=agg_trades_1000pepeusdt_sell[['amount','avg_price']].rename(columns={'amount':'traded'}).add_suffix('_sell')

pepeusdt_ohlc=pepeusdt_ohlc.join(agg_trades_1000pepeusdt_buy).join(agg_trades_1000pepeusdt_sell)



#создание OHLC свечей для dogeusdt


dogeusdt_ohlc = trades_dogeusdt['price'].resample(T).ohlc()

#объем покупок и продаж 

trades_dogeusdt['product']=trades_dogeusdt['price']*trades_dogeusdt['amount']

#cредняя цена покупок

agg_trades_dogeusdt_buy=trades_dogeusdt[trades_dogeusdt['side']=='buy'].groupby(pd.Grouper(freq=T))[['product','amount']].sum()

agg_trades_dogeusdt_buy['avg_price']=agg_trades_dogeusdt_buy['product']/agg_trades_dogeusdt_buy['amount']

agg_trades_dogeusdt_buy=agg_trades_dogeusdt_buy[['amount','avg_price']].rename(columns={'amount':'traded'}).add_suffix('_buy')


#средняя цена продаж


agg_trades_dogeusdt_sell=trades_dogeusdt[trades_dogeusdt['side']=='sell'].groupby(pd.Grouper(freq=T))[['product','amount']].sum()

agg_trades_dogeusdt_sell['avg_price']=agg_trades_dogeusdt_sell['product']/agg_trades_dogeusdt_sell['amount']

agg_trades_dogeusdt_sell=agg_trades_dogeusdt_sell[['amount','avg_price']].rename(columns={'amount':'traded'}).add_suffix('_sell')


dogeusdt_ohlc=dogeusdt_ohlc.join(agg_trades_dogeusdt_buy).join(agg_trades_dogeusdt_sell)

 

del bbo_1000pepeusdt,bbo_dogeusdt,trades_1000pepeusdt,trades_dogeusdt,agg_trades_dogeusdt_buy,agg_trades_dogeusdt_sell,agg_trades_1000pepeusdt_sell,agg_trades_1000pepeusdt_buy




**Задание 2**

Вычисление метрик для каждой стратегии:

*1. PnL (Profit and Loss)*: финансовый результат стратегии

*2. Проторгованный объём (Traded Volume)*: суммарное количество контрактов, которые стратегия купила/продала

*3. Sharpe Ratio*: соотношение среднего дохода к стандартному отклонению доходности

*4. Sortino Ratio*: модифицированное отношение Шарпа с фокусом на негативные отклонения

*5. Максимальная просадка PnL (Max Drawdown)*: наибольшая потеря капитала

*6. Среднее время удержания позиции (Average Holding Time)*: средняя продолжительность, на которую стратегия держит актив

*7. Количество переходов через 0 (Number of Position Flips)*: сколько раз стратегия меняет направление (из покупки в продажу и наоборот)

In [35]:
#считаем среднее скользящее (МА). Если цена выше МА, то . Если цена ниже, то продаем.


#для pepeusdt

pepeusdt_ohlc['MA'] = pepeusdt_ohlc['close'].rolling(window=5).mean()

vol=pepeusdt_ohlc['close'].std()/2 #деление ну просто так (чтобы побольше точек было)

pepeusdt_ohlc['Action']=pepeusdt_ohlc.apply(lambda x: -1 if x['close']<x['MA']-vol else 1 if x['close']> x['MA']+vol else 0,axis=1).fillna(0)

pepeusdt_actions=pepeusdt_ohlc[['Action']]

pepeusdt_ohlc=pepeusdt_ohlc.drop(columns=['MA','Action'])

 

 

#для dogeusdt

dogeusdt_ohlc['MA'] = dogeusdt_ohlc['close'].rolling(window=5).mean()

vol=dogeusdt_ohlc['close'].std()/2 #деление ну просто так (чтобы побольше точек было)

dogeusdt_ohlc['Action']=dogeusdt_ohlc.apply(lambda x: -1 if x['close']<x['MA']-vol else 1 if x['close']> x['MA']+vol else 0,axis=1).fillna(0)

dogeusdt_actions=dogeusdt_ohlc[['Action']]

dogeusdt_ohlc=dogeusdt_ohlc.drop(columns=['MA','Action'])

   

#вторая стратегия:

#считаем корреляцию на скользящем окне. Если корреляция разъезжается, то покупаем/продаем пару
corr_calculator = dogeusdt_ohlc[['close']].add_suffix('_doge').join(pepeusdt_ohlc[['close']].add_suffix('_pepe'))

corr_calculator['corr'] = corr_calculator['close_doge'].rolling(10).corr(corr_calculator['close_pepe'])

avg = corr_calculator['corr'].mean()

vol = corr_calculator['corr'].std()

corr_calculator['Action_doge'] = corr_calculator.apply(lambda x: -1 if x['corr'] < avg - vol else 1 if x['corr'] > avg + vol else 0, axis=1).fillna(0)

corr_calculator['Action_pepe'] = corr_calculator['Action_doge'].apply(lambda x: 1 if x == -1 else -1 if x == 1 else 0)

actions_pair = corr_calculator[['Action_doge', 'Action_pepe']]

del corr_calculator, avg, vol

 

# Функция для симуляции торговли

def trade_simulator(ohlc_df, actions_df, mode='close'):
    
    result = actions_df.copy()
    
    result['OpenPosition'] = 0  #количество открытых контрактов
    
    result['PositionPrice'] = 0  #цена на момент открытия позиции
    
    result['VWAP'] = 0  #объемное взвешенное среднее, рассчитывается для учёта всех сделок в периоде
    
    result['MV'] = 0

    result = result.join(ohlc_df[['close']])
    
    result.reset_index(drop=False, inplace=True)
    
    for r, row in result.iterrows():
        if r == 0:
            continue

        
        prev_row = result.iloc[r - 1]
        
        #позиции и значение не изменяются
        if row['Action'] == 0:
            
            result.at[r, 'OpenPosition'] = prev_row['OpenPosition']
            
            result.at[r, 'PositionPrice'] = prev_row['PositionPrice']
            
            result.at[r, 'VWAP'] = prev_row['VWAP']

        
        #увеличивается открытая позиция, устанавливается новая цена позиции
        elif row['Action'] == 1:
            
            result.at[r, 'OpenPosition'] = prev_row['OpenPosition'] + row['Action']

            result.at[r, 'PositionPrice'] = row['close']
            
            result.at[r, 'VWAP'] = prev_row['VWAP'] + row['Action'] * prev_row['close']
            
        #открытая позиция уменьшается, устанавливается новая цена позиции
        elif row['Action'] == -1:
            
            result.at[r, 'OpenPosition'] = prev_row['OpenPosition'] + row['Action']
            
            result.at[r, 'PositionPrice'] = row['close']
            
            result.at[r, 'VWAP'] = prev_row['VWAP'] + row['Action'] * prev_row['close']
            
    #заполняем пропуски предыдущими непропущенными
    
    result.fillna(method='ffill', inplace=True)
    
    return result





#СТАТИСТИКИ

def calculate_statistics(result_df):

    #рассчет PnL
    result_df['PnL'] = result_df['OpenPosition'] * (result_df['close'] - result_df['PositionPrice'])
    
    #количество контрактов (куплено/продано)
    
    result_df['TradedVolume'] = result_df['OpenPosition'].abs()

'''метрики'''

    #общее значение прибыли/убытка
    pnl = result_df['PnL'].sum()

    #общий объем проторгованных контрактов
    traded_volume = result_df['TradedVolume'].sum()

    #максимальная просадка
    max_drawdown = (result_df['PnL'].cumsum().min() - result_df['PnL'].cumsum().max())


'''Sharpe ratio, Sortino ratio'''    

    #процентное изменение PnL по каждой свече
    
    returns = result_df['PnL'].pct_change().dropna()
    sharpe_ratio = np.mean(returns) / np.std(returns)
    
    #учитывает только риски с убытками
    
    sortino_ratio = np.mean(returns) / np.std(returns[returns < 0])
    
'''average holding time'''
    
    #количество периодов когда есть открытая позиция 
    holding_times = result_df['OpenPosition'].abs().where(result_df['OpenPosition'] != 0).groupby(result_df.index.to_period('D')).size()
    
    average_holding_time = holding_times.mean() if holding_times.size > 0 else 0

    #number of position flips    
    number_of_flips = (result_df['OpenPosition'].diff().abs() > 0).sum()


'''РЕЗУЛЬТАТЫ МЕТРИКИ'''    

    return {
        'PnL': pnl,
        'TradedVolume': traded_volume,
        'SharpeRatio': sharpe_ratio,
        'SortinoRatio': sortino_ratio,
        'MaxDrawdown': max_drawdown,
        'AverageHoldingTime': average_holding_time,
        'NumberOfPositionFlips': number_of_flips
    }


#проверка корректности симулятора
def verify_simulator(ohlc_df, actions_df, true_pnl):
    #вызов функций
    simulated_result = trade_simulator(ohlc_df, actions_df)
    stats = calculate_statistics(simulated_result)
    '''
    стратегия, покупающая и продающая случайное количество контрактов. должна иметь PnL близкий к нулю
    '''
    
    assert np.isclose(stats['PnL'], 0), 

    '''
    стратегия, знающая будущеею. должна иметь высокий PnL
    '''
    
    assert stats['PnL'] > true_pnl, 
    
    print("check")

# Симуляция и расчёт статистик
pepeusdt_result = trade_simulator(pepeusdt_ohlc, pepeusdt_actions)

dogeusdt_result = trade_simulator(dogeusdt_ohlc, actions_pair[['Action_doge']])  # Обратите внимание на использование Action_doge

# Расчёт статистик
pepeusdt_stats = calculate_statistics(pepeusdt_result)

dogeusdt_stats = calculate_statistics(dogeusdt_result)

print("PEPEUSDT Stats:", pepeusdt_stats)

print("DOGEUSDT Stats:", dogeusdt_stats)

# Проверка корректности
true_pnl = 0  # Здесь указывается правильное значение для проверки
verify_simulator(pepeusdt_ohlc, pepeusdt_actions, true_pnl)
verify_simulator(dogeusdt_ohlc, actions_pair[['Action_doge']], true_pnl)



IndentationError: unexpected indent (3195418272.py, line 132)

In [ ]:
'''симуляция'''
pepeusdt_result = trade_simulator(pepeusdt_ohlc, pepeusdt_actions)
dogeusdt_result = trade_simulator(dogeusdt_ohlc, actions_pair[['Action_doge']])

'''подсчет статистик'''

pepeusdt_stats = calculate_statistics(pepeusdt_result)
dogeusdt_stats = calculate_statistics(dogeusdt_result)

'''вывод статистик'''

print("PEPEUSDT Stats:", pepeusdt_stats)
print("DOGEUSDT Stats:", dogeusdt_stats)

'''проверка симуляторов'''

verify_simulator(pepeusdt_ohlc, pepeusdt_actions, true_pnl)
verify_simulator(dogeusdt_ohlc, actions_pair[['Action_doge']], true_pnl)
